<a href="https://colab.research.google.com/github/loicpages/qmII_group_rep/blob/main/Gif_map_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Libraries
import pandas as pd
import PIL
import pylab
import numpy as math
import io
import matplotlib.pyplot as plt
from functools import reduce
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
#Let's install some more
!pip install git+git://github.com/geopandas/geopandas.git
!pip install mapclassify --quiet

In [8]:
#...and import then
import geopandas as gpd
import mapclassify.classifiers as classifiers
from mapclassify import __version__ as mc_version

In [17]:
#let's create our food availibility data frame
x = pd.read_csv('/content/df_final_use.csv')
fa_df = x[['Country', 'Year', 'Food Availibility per capita (kg/cap/year)']]
#And a countries data frame (list of all countries in the world)
cdf = pd.read_csv('/content/countries_world.csv')
fa_df

,Country,Year,Food Availibility per capita (kg/cap/year)
0,Afghanistan,1980,NaN
1,Afghanistan,1981,NaN
2,Afghanistan,1982,NaN
3,Afghanistan,1983,NaN
4,Afghanistan,1984,NaN
...,...,...,...
2579,Zimbabwe,2009,141.317483
2580,Zimbabwe,2010,136.990970
2581,Zimbabwe,2011,139.933312
2582,Zimbabwe,2012,142.318797


In [18]:
new = fa_df.merge(cdf, on = 'Country', how = 'outer')
new.sort_values(by = 'Country', inplace = True)
new['Year'] = new['Year'].astype('Int64')

In [19]:
new.set_index(['Country', 'Year'], inplace = True)
unstacked = new.unstack()
unstacked.sort_values(by = 'Country', inplace = True)

In [20]:
#uploading the world map.
world_map = gpd.read_file(r'/content/World_Map.shp')

In [21]:
#Using a for loop, iterate through the names of the countries in our food availibility data set.
#See, using an if statement, which match with the World data set data and which don't. 
for index, row in unstacked.iterrows():
  if index not in world_map['NAME'].to_list():
    print(index)
  else:
    pass

Brunei
CAR
Congo, Democratic Republic of the
Congo, Republic of the
Céte d'Ivoire
DRCongo
DomRep
East Timor (Timor-Leste)
Iran
Korea, North
Korea, South
Kosovo
Laos
Libya
Macedonia
Moldova
Myanmar (Burma)
North Korea
South Sudan
Syria
Tanzania
The Bahamas
The Gambia
United States of America
Vatican City (Holy See)
Vietnam


In [22]:
#replace the names of the countries that don't match.
world_map.replace('Central African Republic', 'CAR', inplace = True)
world_map.replace('Dominican Republic', 'DomRep', inplace = True)
world_map.replace("Lao People's Democratic Republic", 'Laos', inplace = True)
world_map.replace('Republic of Moldova', 'Moldova', inplace = True)
world_map.replace("Korea, Democratic People's Republic of", 'North Korea', inplace = True)
world_map.replace('United Republic of Tanzania', 'Tanzania', inplace = True)
world_map.replace('Viet Nam', 'Vietnam', inplace = True)
world_map.replace("Cote d'Ivoire", "Céte d'Ivoire", inplace = True)
world_map.replace('Brunei Darussalam', 'Brunei', inplace = True)
world_map.replace('Democratic Republic of the Congo', 'Congo, Democratic Republic of the', inplace = True)
world_map.replace("Congo", 'Congo, Republic of the', inplace = True)
world_map.replace('Timor-Leste', 'East Timor (Timor-Leste)', inplace = True)
world_map.replace('Iran (Islamic Republic of)', 'Iran', inplace = True)
world_map.replace("Korea, Democratic People's Republic of", 'Korea, North', inplace = True)
world_map.replace('Korea, Republic of', 'Korea, South', inplace = True)
world_map.replace('Libyan Arab Jamahiriya', 'Libya', inplace = True)
world_map.replace('The former Yugoslav Republic of Macedonia', 'Macedonia', inplace = True)
world_map.replace('Burma', 'Myanmar (Burma)', inplace = True)
world_map.replace('Sudan', 'South Sudan', inplace = True)
world_map.replace('Syrian Arab Republic', 'Syria', inplace = True)
world_map.replace('Bahamas', 'The Bahamas', inplace = True)
world_map.replace('Gambia', 'The Gambia', inplace = True)
world_map.replace('United States', 'United States of America', inplace = True)
world_map.replace('Holy See (Vatican City)', 'Vatican City (Holy See)', inplace = True)

In [23]:
#check again.
for index, row in unstacked.iterrows():
  if index not in world_map['NAME'].to_list():
    print(index)
  else:
    pass

Central African Republic
Congo
Cote d'Ivoire
DRCongo
Dominican Republic
Gambia
Korea, North
Kosovo
Sudan


In [24]:
#The remaining countries are then those that are twice in our unstacked data frame, that is:
#those that were not matching when merging our data frames with the countries in the world data frame. So:
unstacked.drop(['Central African Republic',
                'DRCongo',
                'Congo',
                "Cote d'Ivoire",
                'Dominican Republic',
                'Gambia',
                'Korea, North',
                'Kosovo',
                'Sudan'],
               inplace = True)

In [25]:
#Final check, everything should be alright now!
for index, row in unstacked.iterrows():
  if index not in world_map['NAME'].to_list():
    print(index)
  else:
    pass

In [26]:
#Merge the world data with our food availibility data
merged = world_map.join(unstacked, on = 'NAME', how = 'right')
merged = merged.fillna(10000)
merged

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,NAME,geometry,"(Food Availibility per capita (kg/cap/year), nan)","(Food Availibility per capita (kg/cap/year), 1980.0)","(Food Availibility per capita (kg/cap/year), 1981.0)","(Food Availibility per capita (kg/cap/year), 1982.0)","(Food Availibility per capita (kg/cap/year), 1983.0)","(Food Availibility per capita (kg/cap/year), 1984.0)","(Food Availibility per capita (kg/cap/year), 1985.0)","(Food Availibility per capita (kg/cap/year), 1986.0)","(Food Availibility per capita (kg/cap/year), 1987.0)","(Food Availibility per capita (kg/cap/year), 1988.0)","(Food Availibility per capita (kg/cap/year), 1989.0)","(Food Availibility per capita (kg/cap/year), 1990.0)","(Food Availibility per capita (kg/cap/year), 1991.0)","(Food Availibility per capita (kg/cap/year), 1992.0)","(Food Availibility per capita (kg/cap/year), 1993.0)","(Food Availibility per capita (kg/cap/year), 1994.0)","(Food Availibility per capita (kg/cap/year), 1995.0)","(Food Availibility per capita (kg/cap/year), 1996.0)","(Food Availibility per capita (kg/cap/year), 1997.0)","(Food Availibility per capita (kg/cap/year), 1998.0)","(Food Availibility per capita (kg/cap/year), 1999.0)","(Food Availibility per capita (kg/cap/year), 2000.0)","(Food Availibility per capita (kg/cap/year), 2001.0)","(Food Availibility per capita (kg/cap/year), 2002.0)","(Food Availibility per capita (kg/cap/year), 2003.0)","(Food Availibility per capita (kg/cap/year), 2004.0)","(Food Availibility per capita (kg/cap/year), 2005.0)","(Food Availibility per capita (kg/cap/year), 2006.0)","(Food Availibility per capita (kg/cap/year), 2007.0)","(Food Availibility per capita (kg/cap/year), 2008.0)","(Food Availibility per capita (kg/cap/year), 2009.0)","(Food Availibility per capita (kg/cap/year), 2010.0)","(Food Availibility per capita (kg/cap/year), 2011.0)","(Food Availibility per capita (kg/cap/year), 2012.0)","(Food Availibility per capita (kg/cap/year), 2013.0)"
30,Afghanistan,"POLYGON ((74.91574 37.23733, 74.83221 37.22041...",10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
3,Albania,"POLYGON ((19.43621 41.02107, 19.45055 41.06000...",10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
1,Algeria,"POLYGON ((2.96361 36.80222, 2.98139 36.80694, ...",10000.0,206.507152,197.960152,196.535700,196.798051,191.743128,201.300172,213.394785,210.742168,218.458482,217.210697,230.949959,239.859144,222.393113,235.393892,207.990992,217.847324,223.274989,218.895104,225.954943,226.818986,228.206245,231.842585,233.939752,240.055480,240.867793,244.302981,246.470993,241.236090,245.880230,249.753737,254.005376,259.098849,263.039750,262.567659
132,Andorra,"POLYGON ((1.78172 42.56996, 1.77472 42.57111, ...",10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
5,Angola,"MULTIPOLYGON (((11.75083 -16.75528, 11.77500 -...",10000.0,134.286

In [ ]:
#Create an emply list that will be incremented with a .png file each loop:
image_list = []

#Create a for loop that:
for years in merged.columns.to_list()[3:36]:
    ax = merged.plot(column = years, cmap ='hot', figsize = (20,20), legend = True, scheme = 'user_defined', 
                     classification_kwds = {'bins':[75,100,125,150,175,200,225,250,275,300]}, edgecolor = 'black', linewidth = 0.5)

#Make a few changes to how the map looks:          
    ax.set_title(years, fontdict = {'fontsize':20}, pad = 12.5)
    ax.set_axis_off()
    ax.get_legend().set_bbox_to_anchor((0.18, 0.5))
    img = ax.get_figure()

#Let us use a buffer and that we can close at a later stage:
    variable = io.BytesIO()
    img.savefig(variable, format = 'png', bbox_inches = 'tight')
    variable.seek(0)
    image_list.append(PIL.Image.open(variable))

#Save images to GIF file
image_list[0].save('Dynamic_map.gif', format = 'GIF',
               append_images = image_list[1:],
               save_all = True, duration = 300,
               loop = 0)
variable.close()